In [1]:
import pandas as pd
import numpy as pd
from UPIBCF import *

In [2]:
raw_rating_df = pd.read_csv('new_ratings_all.txt', sep=':', header=None, names=['uid', 'iid', 'rating', 'date', 'comment_type'], dtype={'uid':np.str, 'iid':np.str})

In [3]:
rating_date_stats = raw_rating_df.groupby(['uid', 'date']).size()

In [4]:
uci = UPIBCF.compute_UCI(raw_rating_df)

In [5]:
urdi = UPIBCF.compute_URDI(raw_rating_df)

In [12]:
urdi.sort_values()

uid
154484332         0.000000
143464675         0.000000
145912870         0.000000
evansey           0.000000
116739424         0.000000
152533709         0.000000
140064558         0.000000
2309445           0.000000
Ada_Abner         0.000000
135455673         0.000000
64251506          0.000000
102027189         0.000000
138576901         0.000000
67132335          0.191833
mio0412           0.263523
56413229          0.263523
orsettp           0.267055
130870119         0.272727
lingqi            0.283466
96210195          0.288275
141561635         0.297647
83602868          0.312694
zmzy              0.313900
80656962          0.333333
52361507          0.343318
83377359          0.343386
83405700          0.344010
46357426          0.346530
59637111          0.349927
132448562         0.353553
                   ...    
51444001          5.332560
46643232          5.355857
jocelyn701        5.397336
4517733           5.414793
41263439          5.418912
etongnet          5.4385

In [7]:
uci

uid
101030945          0.029148
101306532          0.154652
102009494          0.018775
102027189          0.303559
10241572           0.265665
103402492          0.121332
1049890            0.053085
104991310          0.692577
10564710           0.194293
106531709          0.017599
1069770            0.174749
1079180            0.176569
108016742          0.188523
109954597          0.525197
1111383            0.073820
114225619          0.003322
114300394          0.121772
1145037            0.149523
114515320          0.711838
114520027          0.055460
115608638          0.091148
116739424          0.700177
119030590          0.019035
119591426          0.027831
120076971          0.140121
120845613          0.234471
121362410          0.701131
121806653          0.279745
122948297          0.159476
1230022            0.069708
                     ...   
xyiyi              0.126077
yange1991          0.172502
yatouzhu           0.439190
yfqc24pippo        0.493379
yiba            

In [15]:
uci_urdi = pd.concat([uci, urdi], axis=1)

In [16]:
uci_urdi

,0,1
uid,,
101030945,0.029148,3.974599
101306532,0.154652,0.608790
102009494,0.018775,2.585939
102027189,0.303559,0.000000
10241572,0.265665,1.964564
103402492,0.121332,1.905299
1049890,0.053085,1.672248
104991310,0.692577,0.484569
10564710,0.194293,0.515079


In [ ]:
def f(x):
    